In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb


In [3]:
assets=['PETR4.SA','ABEV3.SA','CIEL3.SA','ITUB4.SA','^BVSP']
portfolio=pd.DataFrame()

In [4]:
import datetime
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 6, 15)

In [5]:
for t in assets:   
    portfolio[t]=wb.DataReader(t,data_source='yahoo',start='2012-1-1',end='2019-6-20')['Adj Close']

sec_ret=np.log(portfolio/portfolio.shift(1))


In [6]:
sec_ret.tail()

,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA,^BVSP
Date,,,,,
2019-06-14,-0.004425,-0.011835,-0.009050,0.000294,-0.007459
2019-06-17,0.001846,0.009029,0.003026,-0.001178,-0.004262
2019-06-18,0.012464,0.013947,0.019447,0.025590,0.018079
2019-06-19,0.002547,0.008276,0.014706,0.015668,0.009003
2019-06-21,0.027242,0.013643,0.060882,0.022360,0.016905


## Matriz de covariância

In [7]:
# Vamos assumir 250 dias ao ano negociados.
covariance=sec_ret.cov()*250
covariance


,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA,^BVSP
PETR4.SA,0.233129,0.032836,0.042000,0.084769,0.083534
ABEV3.SA,0.032836,0.052942,0.022316,0.027617,0.025215
CIEL3.SA,0.042000,0.022316,0.106148,0.032619,0.029866
ITUB4.SA,0.084769,0.027617,0.032619,0.091786,0.055732
^BVSP,0.083534,0.025215,0.029866,0.055732,0.050604


## Índice Beta: Como é Calculado?

A fórmula do Índice Beta é bem simples:

Beta = Covariância entre o Retorno do Ativo e do Mercado / Variância do Retorno do Mercado

In [8]:
marketcov = covariance.iloc[:,-1]
market_var=sec_ret['^BVSP'].var()*250

In [9]:
marketcov

PETR4.SA    0.083534
ABEV3.SA    0.025215
CIEL3.SA    0.029866
ITUB4.SA    0.055732
^BVSP       0.050604
Name: ^BVSP, dtype: float64

In [10]:
market_var

0.05060395356495615

In [11]:
betam = marketcov/market_var
betam

PETR4.SA    1.650731
ABEV3.SA    0.498275
CIEL3.SA    0.590182
ITUB4.SA    1.101340
^BVSP       1.000000
Name: ^BVSP, dtype: float64

In [12]:

media = sec_ret.mean()*250
media[-1]
#Premio_bvsp = 

0.07502046478803245

In [14]:
# Expected Return
premio = sec_ret['^BVSP'].mean()*250 - 0.065
Exp_ret= 0.065+betam*premio
Exp_ret

PETR4.SA    0.081541
ABEV3.SA    0.069993
CIEL3.SA    0.070914
ITUB4.SA    0.076036
^BVSP       0.075020
Name: ^BVSP, dtype: float64

## Índice de Sharpe: Como é Calculado?

IS = ( Ri -Rf ) / (σi )

Onde:

    IS = Índice de Sharpe
    Ri = Retorno do Ativo
    Rf = Retorno Livre de Riso ( Risk-Free )
    σi = Risco do Ativo ( a letra grega sigma representa volatilidade )


In [15]:
# Calculation os Sharpe ratio

Rf = 0.065   #SELIC witch risk free

sharpes =[]

sigma = (sec_ret.std()*(250**0.5))

sharpe=(Exp_ret-Rf)/sigma
sharpe

PETR4.SA    0.034258
ABEV3.SA    0.021700
CIEL3.SA    0.018152
ITUB4.SA    0.036427
^BVSP       0.044545
dtype: float64

In [16]:
Exp_ret

PETR4.SA    0.081541
ABEV3.SA    0.069993
CIEL3.SA    0.070914
ITUB4.SA    0.076036
^BVSP       0.075020
Name: ^BVSP, dtype: float64

In [17]:
Exp_ret-Rf

PETR4.SA    0.016541
ABEV3.SA    0.004993
CIEL3.SA    0.005914
ITUB4.SA    0.011036
^BVSP       0.010020
Name: ^BVSP, dtype: float64

In [18]:
sigma

PETR4.SA    0.482835
ABEV3.SA    0.230092
CIEL3.SA    0.325804
ITUB4.SA    0.302962
^BVSP       0.224953
dtype: float64

## CAPM

ERi = Expected return of investment

Rf = Risk-free rate

βi = Beta of the investment

ERm = Expected return of market

(ERm - Rf) = Market risk premium

ERi = Rf + βi(ERm - Rf)


In [45]:
import scipy.optimize as sco


# portfolios = sco.minimize(, [0.5, 0.1, 0.2, 0.2], )


# def statistics():
#     pret = sec_ret[-250:]
#     pvol = allocations
#     return np.array([pret, pvol, pret/pvol])


def min_risk(shares):
    risk = np.dot(marketcov[:4]*250, np.power(shares, 2))
    return np.array(risk)

min_risk([0.4, 0.3, 0.2, 0.4])

array(6.43661261)

In [39]:
shares = [0.4, 0.3, 0.2, 0.1]
risk = np.dot(shares, np.dot(marketcov[:4]*250, shares))
np.dot(risk, [1,1,1,1])

13.131034593052949

In [40]:
np.dot(marketcov[:4]*250, shares)/250

0.0525241383722118

In [53]:
min_ret = .07
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1}, {'type': 'ineq', 'fun': lambda x: np.dot(x, Exp_ret[:4])-min_ret})
bnds = [(0, 1)]*(len(media)-1)

opts = sco.minimize(min_risk, (len(media)-1)*[(len(media)-1)**-1,], method='SLSQP', bounds = bnds, constraints=cons)
opts['x']

array([0.11616227, 0.3848397 , 0.32489025, 0.17410778])

In [52]:
min_ret = .08
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1}, {'type': 'ineq', 'fun': lambda x: np.dot(x, Exp_ret[:4])-min_ret})

opts = sco.minimize(min_risk, (len(media)-1)*[(len(media)-1)**-1,], method='SLSQP', bounds = bnds, constraints=cons)
opts['x']

array([0.72006369, 0.        , 0.        , 0.27993631])